# USAID-Kenya-Sentiment-Analysis

### Extracting Data From X/Tweeter API (Tweepy)

In [1]:
!pip install --user tweepy



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import tweepy

# # Bearer token given
# BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAFIL2QEAAAAAbjIT6gj7oGnos9zXrJRrAK09dJM%3DQ7CzTAow8k1XIFJaAFRGjKuQDFqBjmPZkyuj9CNrgSvGCuvlLg'

# # Initialize the Twitter client
# client = tweepy.Client(bearer_token=BEARER_TOKEN)

# # Query includes retweets, multiple keywords, and allows any language
# query = '(USAID OR "foreign aid" OR "funding cuts" OR "cut USAID funding" OR "cutting USAID" OR "halted USAID support" OR "cut funding" OR "usaid budget cuts" OR "NGO funding" OR "Africa aid" OR "USAID gets shut" OR "Trump cuts usaid" OR "USAID money" OR "aid withdrawal" OR "end of USAID" OR "Trump Cuts Aid To Africa" OR NGOs funding) (Kenya OR Nairobi)'

# # Fetch recent tweets (up to 100 at a time, max 7 days old)
# response = client.search_recent_tweets(query=query, max_results=100)

# # Print the tweets
# if response.data:
#     for tweet in response.data:
#         print(tweet.text)
# else:
#     print("No tweets found.")

## Extracting Data From Reddit API (PRAW)

In [3]:
!pip install praw



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\commands\install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(se

In [4]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API client
reddit = praw.Reddit(
    client_id='f6MuemGIIpuv8csr3CpuzQ',
    client_secret='rSPUbtIsCzfYWW3Lf9mnR330gX7FCA',
    user_agent='USAID-Ke-Prac'
)

# Keywords related to USAID, Kenya, Trump, and aid cuts
keywords = [
    "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya", 
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors","end of USAID","Trump Cuts Aid To Africa","NGOs"
]

relevant_subreddits = ["Kenya","Nairobi"]

# the cutoff date
CUTOFF_DATE = datetime(2025, 1, 1)  

def search_reddit(subreddits, limit=100):
    posts = []
    
    for subreddit_name in subreddits:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            for submission in subreddit.search("USAID OR Kenya OR aid", limit=limit, sort="new"):
                post_date = datetime.utcfromtimestamp(submission.created_utc)
                
                # Skip posts before the cutoff date
                if post_date < CUTOFF_DATE:
                    continue
                
                title_lower = submission.title.lower()
                selftext_lower = submission.selftext.lower()
                matched_keywords = [kw for kw in keywords if kw.lower() in title_lower or kw.lower() in selftext_lower]
                
                if matched_keywords:
                    posts.append({
                        'subreddit': subreddit_name,
                        'keyword': matched_keywords[0],
                        'title': submission.title,
                        'text': submission.selftext,
                        'date_posted': post_date.strftime('%Y-%m-%d %H:%M:%S'),
                        'score': submission.score,
                        'url': submission.url,
                        'permalink': f"https://reddit.com{submission.permalink}",
                        'upvotes': submission.score,
                        'comments': submission.num_comments
                    })
        except Exception as e:
            print(f"Skipped r/{subreddit_name}: {e}")
    
    return pd.DataFrame(posts)

df_reddit = search_reddit(relevant_subreddits, limit=300)
print(f"Found {len(df_reddit)} relevant posts after {CUTOFF_DATE}")
df_reddit.head(50)

Found 2 relevant posts after 2025-01-01 00:00:00


,subreddit,keyword,title,text,date_posted,score,url,permalink,upvotes,comments
0,Nairobi,USAID,Theres always someone out there worse off than...,I was talking to my South Sudanese friend yest...,2025-05-12 23:28:50,81,https://www.reddit.com/r/nairobi/comments/1kl7...,https://reddit.com/r/nairobi/comments/1kl7bcm/...,81,53
1,Nairobi,aid withdrawal,What steps are you taking to keep your busines...,"Let's face it, since last year August; a lot o...",2025-04-14 09:54:50,6,https://www.reddit.com/r/nairobi/comments/1jyv...,https://reddit.com/r/nairobi/comments/1jyv2tj/...,6,4


In [5]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API Setup
reddit = praw.Reddit(
    client_id='f6MuemGIIpuv8csr3CpuzQ',
    client_secret='rSPUbtIsCzfYWW3Lf9mnR330gX7FCA',
    user_agent='USAID-Kenya-Aid-Analysis'
)

# Kenya-Specific Keywords
keywords = [
    "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya", 
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors", "end of USAID", "Trump Cuts Aid To Africa", "NGOs"
]

# Only Kenya-focused subreddits
relevant_subreddits = ["Kenya", "Nairobi"]  

# Cutoff date (March 2025)
CUTOFF_DATE = datetime(2025, 1, 1)  # Adjust to exact announcement date if needed

def search_reddit_kenya(subreddits, limit=100):
    posts = []
    
    for subreddit_name in subreddits:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            # Search for "Kenya + aid" in subreddits
            for submission in subreddit.search("Kenya AND (USAID OR aid OR funding)", limit=limit, sort="new"):
                post_date = datetime.utcfromtimestamp(submission.created_utc)
                
                # Skip posts before the cutoff date
                if post_date < CUTOFF_DATE:
                    continue
                
                # Check for Kenya-specific keywords in title/text
                content = f"{submission.title} {submission.selftext}".lower()
                matched_keywords = [kw for kw in keywords if kw.lower() in content]
                
                if matched_keywords:
                    posts.append({
                        'subreddit': subreddit_name,
                        'keyword': ", ".join(matched_keywords),
                        'title': submission.title,
                        'text': submission.selftext,
                        'date_posted': post_date.strftime('%Y-%m-%d'),
                        'upvotes': submission.score,
                        'comments': submission.num_comments,
                        'url': submission.url,  # Direct URL to linked content
                        'permalink': f"https://reddit.com{submission.permalink}"  # Link to Reddit discussion
                    })
        except Exception as e:
            print(f"Error in r/{subreddit_name}: {e}")
    
    return pd.DataFrame(posts)

# Run search
df_kenya = search_reddit_kenya(relevant_subreddits, limit=200)
print(f"Found {len(df_kenya)} Kenya-focused posts after {CUTOFF_DATE.date()}")
df_kenya.head(70)

Found 32 Kenya-focused posts after 2025-01-01


,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
0,Kenya,donors,Dear Gentle Readers,Good morning my gentle readers\n\nToday I have...,2025-05-30,52,74,https://www.reddit.com/r/Kenya/comments/1kyxeb...,https://reddit.com/r/Kenya/comments/1kyxebr/de...
1,Kenya,USAID,USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,2025-05-14,11,32,https://www.reddit.com/r/Kenya/comments/1kmhn8...,https://reddit.com/r/Kenya/comments/1kmhn87/us...
2,Kenya,Trump cuts,Bill Gates 'horrified' by Trump cuts to US aid...,,2025-05-08,1,1,https://www.semafor.com/article/05/08/2025/bil...,https://reddit.com/r/Kenya/comments/1khpakg/bi...
3,Kenya,"foreign aid, foreign aid",Foreign aid/Philanthropy,I see the aid or philanthropic activities that...,2025-04-27,1,0,https://www.reddit.com/r/Kenya/comments/1k91v5...,https://reddit.com/r/Kenya/comments/1k91v58/fo...
4,Kenya,USAID,"USAID left a month ago, do we have ARVs in Kenya?",Someone on a different group (different websit...,2025-04-15,3,5,https://www.reddit.com/r/Kenya/comments/1jzrn2...,https://reddit.com/r/Kenya/comments/1jzrn2s/us...
5,Kenya,USAID,Classism in r/Kenya and r/nairobi,The classism I'm seeing in both subs is a good...,2025-04-07,162,96,https://www.reddit.com/r/Kenya/comments/1jtcvb...,https://reddit.com/r/Kenya/comments/1jtcvbx/cl...
6,Kenya,USAID,Economy,For the experts in matters economy and finance...,2025-04-06,1,15,https://www.reddit.com/r/Kenya/comments/1jsyty...,https://reddit.com/r/Kenya/comments/1jsytyp/ec...
7,Kenya,USAID,EX-USAID people!! Let's talk,Are you still in contact with the organisation...,2025-04-05,2,0,https://www.reddit.com/r/Kenya/comments/1jsb14...,https://reddit.com/r/Kenya/comments/1jsb149/ex...
8,Kenya,"foreign aid, foreign aid",Kibaki ALSO failed us,\nThere is a tendency to over-exaggerate the p...,2025-04-01,120,124,https://www.reddit.com/r/Kenya/comments/1jojl2...,https://reddit.com/r/Kenya/comments/1jojl2f/ki...
9,Kenya,"foreign aid, foreign aid",Is Data the missing link for Kenya's entrepren...,Africa’s economic future is being redefined by...,2025-03-30,2,5,https://www.reddit.com/r/Kenya/comments/1jnfss...,https://reddit.com/r/Kenya/comments/1jnfssf/is...


In [6]:
df_kenya.sample(30)

,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
30,Nairobi,USAID,The USAID effect,The whole USAID issue has dealt a big blow to ...,2025-02-05,2,5,https://www.reddit.com/r/nairobi/comments/1iif...,https://reddit.com/r/nairobi/comments/1iifq41/...
5,Kenya,USAID,Classism in r/Kenya and r/nairobi,The classism I'm seeing in both subs is a good...,2025-04-07,162,96,https://www.reddit.com/r/Kenya/comments/1jtcvb...,https://reddit.com/r/Kenya/comments/1jtcvbx/cl...
6,Kenya,USAID,Economy,For the experts in matters economy and finance...,2025-04-06,1,15,https://www.reddit.com/r/Kenya/comments/1jsyty...,https://reddit.com/r/Kenya/comments/1jsytyp/ec...
29,Nairobi,USAID,Karibu na mzuri! A day in the life of a job se...,Karibu na mzuri! A day in the life of a job se...,2025-02-07,7,0,https://www.reddit.com/r/nairobi/comments/1ijp...,https://reddit.com/r/nairobi/comments/1ijpglc/...
14,Kenya,USAID,The Israel-Gaza War: A Symptom of a Looming Gl...,"At first glance, BRICS, the rise of the AfD, T...",2025-02-26,3,10,https://www.reddit.com/r/Kenya/comments/1iygom...,https://reddit.com/r/Kenya/comments/1iygomq/th...
0,Kenya,donors,Dear Gentle Readers,Good morning my gentle readers\n\nToday I have...,2025-05-30,52,74,https://www.reddit.com/r/Kenya/comments/1kyxeb...,https://reddit.com/r/Kenya/comments/1kyxebr/de...
17,Kenya,USAID,These seven African countries will be hit the ...,,2025-02-12,2,0,https://www.semafor.com/article/02/12/2025/cou...,https://reddit.com/r/Kenya/comments/1intuao/th...
26,Nairobi,USAID,Theres always someone out there worse off than...,I was talking to my South Sudanese friend yest...,2025-05-12,80,53,https://www.reddit.com/r/nairobi/comments/1kl7...,https://reddit.com/r/nairobi/comments/1kl7bcm/...
20,Kenya,USAID,USAID woes,many people don't know how USAID made many Ken...,2025-02-07,3,9,https://www.reddit.com/r/Kenya/comments/1ijyka...,https://reddit.com/r/Kenya/comments/1ijyka5/us...
16,Kenya,"USAID, NGOs","Finally, it's gets deeper. I lost my job","With the recent USAID cuts, I knew we might g...",2025-02-12,187,74,https://www.reddit.com/r/Kenya/comments/1iny5w...,https://reddit.com/r/Kenya/comments/1iny5w8/fi...


## Extracting Data From News API (NewsAPI.org)

In [7]:
!pip install requests


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\commands\install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(se

In [16]:
import requests
import pandas as pd
import time

# My NewsAPI key
API_KEY = 'e11a1726c9c1470ea9abc0eead154d6c'

# Base URL
BASE_URL = 'https://newsapi.org/v2/everything'

# Keywords to search
keywords = [
    "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya",
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors", "end of USAID", "Trump Cuts Aid To Africa", "NGOs"
]


all_articles = []

# Loop through each keyword
for keyword in keywords:
    print(f"Fetching articles for: {keyword}")
    
    params = {
        'q': keyword,
        'apiKey': API_KEY,
        'language': 'en',
        'sortBy': 'relevancy',
        'pageSize': 100,
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        for article in articles:
            all_articles.append({
                'keyword': keyword,
                'source': article['source']['name'],
                'title': article['title'],
                'description': article['description'],
                'url': article['url'],
                'publishedAt': article['publishedAt']
            })
    else:
        print(f"Error for keyword '{keyword}':", response.status_code, response.text)
    
    # To avoid rate limiting
    time.sleep(1.5)

# Convert to DataFrame
df = pd.DataFrame(all_articles)
df.sample (50)

Fetching articles for: USAID
Fetching articles for: foreign aid
Fetching articles for: funding cuts
Fetching articles for: aid budget
Fetching articles for: Trump Kenya
Fetching articles for: USAID Kenya
Fetching articles for: cut aid
Fetching articles for: budget cuts
Fetching articles for: NGO funding
Fetching articles for: Africa aid
Fetching articles for: USAID funding
Fetching articles for: foreign aid
Fetching articles for: Trump cuts
Fetching articles for: USAID money
Fetching articles for: aid withdrawal
Fetching articles for: State Department cuts
Fetching articles for: donors
Fetching articles for: end of USAID
Fetching articles for: Trump Cuts Aid To Africa
Fetching articles for: NGOs


,keyword,title,description,url,publishedAt,source
483,USAID Kenya,Willingness-to-pay and demand for sanitation p...,Open defecation practices persist in some reso...,https://journals.plos.org/water/article?id=10....,2025-05-06T14:00:00Z,Plos.org
1485,donors,Science Tells Us the U.S. Is Heading toward a ...,The red flags abound—political research tells ...,https://www.scientificamerican.com/article/sci...,2025-05-14T11:30:00Z,Scientific American
763,NGO funding,Prince Harry Urged to Quit Charity Where Abuse...,Prince Harry is a board member at African Park...,https://www.newsweek.com/prince-harry-quit-afr...,2025-05-12T15:00:50Z,Newsweek
231,funding cuts,Trump tried to shutter Radio Free Europe. The ...,EU officials say the broadcaster for years has...,https://www.npr.org/2025/05/21/g-s1-68363/trum...,2025-05-22T03:50:19Z,NPR
569,cut aid,Trump Housing Assistance Cuts Would Hurt Peopl...,"The single largest portion of a big, planned c...",https://www.forbes.com/sites/eriksherman/2025/...,2025-05-07T01:15:34Z,Forbes
1497,donors,Republicans plan to tax US college endowments:...,Republicans say their new bill holds 'woke' un...,https://www.aljazeera.com/news/2025/5/30/repub...,2025-05-30T04:45:28Z,Al Jazeera English
1275,aid withdrawal,Trump’s Liquidation of U.S Global Leadership,Some geopolitical aspects of how Trump's mazim...,https://www.nakedcapitalism.com/2025/05/trumps...,2025-05-27T07:00:05Z,Nakedcapitalism.com
965,foreign aid,US aid kept many hungry Somali children alive....,The U.S. Agency for International Development ...,https://www.yahoo.com/news/us-aid-kept-many-hu...,2025-05-27T03:01:23Z,Yahoo Entertainment
1191,USAID money,President Trump Is Demolishing America’s Globa...,Alfred McCoy\n\n\n\nTrump’s sharp cuts to huma...,https://www.thenation.com/?post_type=article&p...,2025-06-03T09:00:00Z,Thenation.com
929,USAID funding,Radio Free Asia reporters sought safety in U.S...,"For decades, U.S. presidents proclaimed a miss...",https://www.cbc.ca/news/world/radio-free-asia-...,2025-05-29T08:00:00Z,CBC News
